In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install "unsloth==2025.2.4" vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-15 01:03:22 __init__.py:190] Automatically detected platform cuda.


In [3]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/phi-4-unsloth-bnb-4bit
==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Unsloth: vLLM loading unsloth/phi-4-bnb-4bit with actual GPU utilization = 69.52%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.28 GB. Also swap space = 2 GB.
WARNING 02-15 01:11:16 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-15 01:11:29 config.py:542] This model supports multiple tasks: {'score', 'classify', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection'], 'llm_int8_threshold': 6.0}
INFO 02-15 01:11:29 

tokenizer_config.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.15M [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.2.4 patched 40 layers with 0 QKV layers, 0 O layers and 40 MLP layers.


In [8]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

def generate_reasoning(question: str, answer: str) -> str:
    """
    Generates reasoning for a given question and answer in the specified XML format.
    """
    reasoning = f"""
<reasoning>
To solve the problem, let's break it down step by step:

1. **Understand the problem**: {question}
2. **Identify the key components**: [Explain the key components of the problem here.]
3. **Perform calculations**: [Show the calculations or logical steps here.]
4. **Verify the solution**: [Double-check the calculations or logic to ensure correctness.]

Based on the above steps, the answer is derived as follows:
</reasoning>
<answer>
{answer}
</answer>
"""
    return reasoning.strip()

# Load dataset with reasoning
def get_gsm8k_with_reasoning(split="train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split]  # type: ignore
    data = data.map(lambda x: {  # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer']),
        'reasoning': generate_reasoning(x['question'], extract_hash_answer(x['answer']))
    })  # type: ignore
    return data  # type: ignore

dataset = get_gsm8k_with_reasoning()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion['content'] for completion in completions] # Access 'content' directly from the dictionary
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-' * 20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    # completions is a list of dictionaries, access 'content' directly
    responses = [completion["content"] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    # completions is a list of dictionaries, access 'content' directly
    responses = [completion["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    # completions is a list of dictionaries, access 'content' directly
    responses = [completion["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1]) * 0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1) * 0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    # completions is a list of dictionaries, access 'content' directly
    contents = [completion["content"] for completion in completions]
    return [count_xml(c) for c in contents]

# Example usage
if __name__ == "__main__":
    # Print the first example from the dataset
    example = dataset[0]
    print("Prompt:", example['prompt'])
    print("Answer:", example['answer'])
    print("Reasoning:", example['reasoning'])

    # Test reward functions
    prompts = [example['prompt']]
    completions = [{'role': 'assistant', 'content': example['reasoning']}]
    answers = [example['answer']]

    print("Correctness Reward:", correctness_reward_func(prompts, completions, answers))
    print("Format Reward:", strict_format_reward_func(completions))

Prompt: [{'content': '\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n', 'role': 'system'}, {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'role': 'user'}]
Answer: 72
Reasoning: <reasoning>
To solve the problem, let's break it down step by step:

1. **Understand the problem**: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
2. **Identify the key components**: [Explain the key components of the problem here.]
3. **Perform calculations**: [Show the calculations or logical steps here.]
4. **Verify the solution**: [Double-check the calculations or logic to ensure correctness.]

Based on the above steps, the answer is derived as follows:
</reasoning>
<answer>
72
</answer>
-------------------- Question:
Natalia sold cl

In [9]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


In [14]:
def reward_wrapper(reward_func):
    """Wraps a reward function to ensure it receives completions as a list of dictionaries."""
    def wrapped_reward_func(completions, **kwargs):
        # Ensure completions is always a list of dictionaries
        completions = [{'content': c} if isinstance(c, str) else c for c in completions]
        # Check if each completion has a 'content' key before accessing it
        completions = [c for c in completions if isinstance(c, dict) and 'content' in c]

        # Print the number of completions before applying the reward function
        print(f"Number of completions received by {reward_func.__name__}: {len(completions)}")

        # Remove 'prompts' from kwargs if it exists to avoid conflict
        kwargs.pop('prompts', None)

        rewards = reward_func(completions=completions, **kwargs) # Pass completions as keyword argument

        # Print the rewards returned by the reward function
        print(f"Rewards returned by {reward_func.__name__}: {rewards}")

        # Ensure that the reward function returns a list of the correct length
        if len(rewards) != training_args.num_generations:
            print(f"Warning: {reward_func.__name__} returned {len(rewards)} rewards, but {training_args.num_generations} were expected.")
            rewards = rewards + [0.0] * (training_args.num_generations - len(rewards))  # Pad with zeros if necessary

        return rewards
    return wrapped_reward_func

In [17]:
def reward_wrapper(reward_func):
    """Wraps a reward function to ensure it receives completions as a list of dictionaries."""
    def wrapped_reward_func(completions, **kwargs):
        # Ensure completions is always a list of dictionaries
        completions = [{'content': c} if isinstance(c, str) else c for c in completions]
        # Check if each completion has a 'content' key before accessing it
        completions = [c for c in completions if isinstance(c, dict) and 'content' in c]

        # Print the number of completions before applying the reward function
        print(f"Number of completions received by {reward_func.__name__}: {len(completions)}")

        # Pass 'prompts' and 'answer' as keyword arguments if the reward function expects them
        if reward_func.__name__ == 'correctness_reward_func':
            rewards = reward_func(completions=completions, prompts=kwargs.get('prompts'), answer=kwargs.get('answer'))
        else:
            # Remove 'prompts' from kwargs if it exists to avoid conflict for other reward functions
            kwargs.pop('prompts', None)
            rewards = reward_func(completions=completions, **kwargs) # Pass completions as keyword argument

        # Print the rewards returned by the reward function
        print(f"Rewards returned by {reward_func.__name__}: {rewards}")

        # Ensure that the reward function returns a list of the correct length
        if len(rewards) != training_args.num_generations:
            print(f"Warning: {reward_func.__name__} returned {len(rewards)} rewards, but {training_args.num_generations} were expected.")
            rewards = rewards + [0.0] * (training_args.num_generations - len(rewards))  # Pad with zeros if necessary

        return rewards
    return wrapped_reward_func

In [19]:
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """
    Calculates the correctness reward by comparing extracted answers with the ground truth.

    Args:
        prompts: A list of dictionaries representing the prompts. Each dictionary should have a 'content' key.
        completions: A list of dictionaries representing the completions. Each dictionary should have a 'content' key.
        answer: A list or a string representing the correct answer.

    Returns:
        A list of floats representing the correctness reward for each completion.
    """
    responses = [completion['content'] for completion in completions]
    q = prompts[0][-1]['content'] if prompts and prompts[0] and len(prompts[0]) > 0 and isinstance(prompts[0][-1], dict) and 'content' in prompts[0][-1] else '' # Access 'content' safely
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Handle the case where answer is not a list or is empty
    if not isinstance(answer, list) or not answer:
        print(f"Warning: answer is not a list or is empty: {answer}")
        return [0.0] * len(responses)  # Return 0 reward for all responses if answer is invalid

    print('-' * 20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")

    # Compare extracted responses with the correct answer (handling potential type mismatch)
    return [2.0 if str(r) == str(a) else 0.0 for r, a in zip(extracted_responses, answer)]

In [21]:
def reward_wrapper(reward_func):
    """Wraps a reward function to ensure it receives completions as a list of dictionaries."""
    def wrapped_reward_func(completions, **kwargs):
        # ... (Existing code) ...

        # Pass 'prompts' and 'answer' as keyword arguments if the reward function expects them
        if reward_func.__name__ == 'correctness_reward_func':
            # Ensure answer is a list and is not empty
            answer = kwargs.get('answer')
            if not isinstance(answer, list) or not answer:
                print(f"Warning: answer is not a list or is empty: {answer}. Returning 0 reward.")
                return [0.0] * training_args.num_generations

            rewards = reward_func(completions=completions, prompts=kwargs.get('prompts'), answer=answer)
        # ... (Existing code) ...

        return rewards
    return wrapped_reward_func

In [23]:
def reward_wrapper(reward_func):
    """Wraps a reward function to ensure it receives completions as a list of dictionaries."""
    def wrapped_reward_func(completions, **kwargs):
        # ... (Existing code) ...

        # Pass 'prompts' and 'answer' as keyword arguments if the reward function expects them
        if reward_func.__name__ == 'correctness_reward_func':
            # Ensure answer is a list and is not empty
            answer = kwargs.get('answer')
            # This is the modified line to handle the string '72'
            answer = [[answer[0]]] if isinstance(answer, list) and isinstance(answer[0], str) else answer
            if not isinstance(answer, list) or not answer or not answer[0]:  # Added check for answer[0]
                print(f"Warning: answer is not a list or is empty: {answer}. Returning 0 reward.")
                return [0.0] * training_args.num_generations

            rewards = reward_func(completions=completions, prompts=kwargs.get('prompts'), answer=answer)
        # ... (Existing code) ...

        return rewards
    return wrapped_reward_func

In [24]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        reward_wrapper(xmlcount_reward_func),
        reward_wrapper(soft_format_reward_func),
        reward_wrapper(strict_format_reward_func),
        reward_wrapper(int_reward_func),
        reward_wrapper(correctness_reward_func),
    ],
    args=training_args,
    train_dataset=dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 100
 "-____-"     Number of trainable parameters = 44,236,800


UnboundLocalError: cannot access local variable 'rewards' where it is not associated with a value

In [25]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Which is bigger? 9.11 or 9.9?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.80s/it, est. speed input: 1.52 toks/s, output: 9.36 toks/s]


"9.11 is bigger than 9.9. When comparing decimal numbers, you look at the digits from left to right. Both numbers have a 9 before the decimal point, so you then compare the digits after the decimal point. The number 9.11 has a 1 in the tenths place, while 9.9 has a 9 in the tenths place. To make the comparison easier, you can think of 9.9 as 9.90. Now, it's clear that 9.11 (which is the same as 9.110) is greater than 9.90."

In [26]:
model.save_lora("grpo_saved_lora")

In [27]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Which is bigger? 9.11 or 9.9?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:19<00:00, 19.32s/it, est. speed input: 2.43 toks/s, output: 8.90 toks/s]


'<reasoning>\nTo determine which number is bigger, we compare 9.11 and 9.9 by looking at their decimal values.\n\n1. **Compare the whole number part:**  \n   Both numbers have the same whole number part, which is 9. So, we move to the decimal part.\n\n2. **Compare the tenths place:**  \n   - 9.11 has a 1 in the tenths place.\n   - 9.9 has a 9 in the tenths place.\n\nSince 9 is greater than 1, 9.9 is already greater than 9.11 at this point, without needing to look further.\n\nTherefore, 9.9 is bigger than 9.11.\n</reasoning>\n\n<answer>\n9.9 is bigger than 9.11.\n</answer>'

In [28]:
print(output)

<reasoning>
To determine which number is bigger, we compare 9.11 and 9.9 by looking at their decimal values.

1. **Compare the whole number part:**  
   Both numbers have the same whole number part, which is 9. So, we move to the decimal part.

2. **Compare the tenths place:**  
   - 9.11 has a 1 in the tenths place.
   - 9.9 has a 9 in the tenths place.

Since 9 is greater than 1, 9.9 is already greater than 9.11 at this point, without needing to look further.

Therefore, 9.9 is bigger than 9.11.
</reasoning>

<answer>
9.9 is bigger than 9.11.
</answer>


In [29]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [30]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
# Instead of:
# save_pretrained_gguf

# Use:
model.save_pretrained_gguf("persadian_14B-GRPO", tokenizer)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 0.91 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [21:26<00:00, 32.15s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving persadian_14B-GRPO/pytorch_model-00001-of-00006.bin...
Unsloth: Saving persadian_14B-GRPO/pytorch_model-00002-of-00006.bin...


In [6]:
!pip install "unsloth==2025.2.4" vllm # Re-add this line to install the 'unsloth' package
!pip install --upgrade pillow # This also installs dependencies
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b # trl is a dependency of unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.

  Cloning https://github.com/huggingface/trl.git (to revision e95f9fb74a3c3647b86f251b7e230ec51c64b72b) to /tmp/pip-req-build-wyd5y8b1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-wyd5y8b1
  Running command git rev-parse -q --verify 'sha^e95f9fb74a3c3647b86f251b7e230ec51c64b72b'
  Running command git fetch -q https://github.com/huggingface/trl.git e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Running command git checkout -q e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Resolved https://github.com/huggingface/trl.git to commit e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.15.0.dev0-py3-none-any.whl size=314513 sha256=ef8411710b28fa54a6c781dd1d24d6e74d0e8436b8bf828efefbf6450066e51e
  Stored in directory: /root/.cache/pip/wheels/2a/b7/04/e2896a98a79e6b4eca45729

In [7]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel) # Remove the extra indentation
# ... (Other code for loading and initializing model) ...

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [8]:
# Assuming 'model' is an instance of FastLanguageModel
# And you have your Hugging Face token
model.push_to_hub_gguf("hf/model", tokenizer, token = "YOUR_TOKEN") # Replace hf/model and YOUR_TOKEN

NameError: name 'model' is not defined